<a href="https://colab.research.google.com/github/HimashiRathnayake/Hate-Speech-Humor-Detection/blob/branch-1/Annotation/Inter_Annotation_Agreement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Parameters**

In [238]:
tag_set = "Sentiment" #@param ["Sentiment","Humor","Aspect","HateSpeech","LanguageID"]
sentences_no = 156 #{allow-input: true} #156, 1349
annotators = ['Janani','Raveesha','Himashi']

if (tag_set=="Sentiment"):
  tags = ['Positive', 'Negative','Neutral','Conflict'];
elif (tag_set=="Humor"):
  tags = ['Humorous','Non-humorous'];
elif (tag_set=="HateSpeech"):
 tags = ['Hate-Inducing', 'Abusive', 'Not offensive'];
elif (tag_set=="Aspect"):
  tags = ['Billing or price','Customer service','Data','Network','None','Package','Service or product'];
elif (tag_set=="LanguageID"):
  tags = ['Sinhala','English','Sin-Eng','Eng-Sin','Unknown','name','Mixed'];

**Read UIMA CAS XMI File to a matrix**

In [239]:
!pip install dkpro-cassis > /dev/null

In [240]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [242]:
from cassis import *
import numpy as np

tags_no = len(tags)
matrix_to_kappa = np.zeros((sentences_no, tags_no), dtype=int)

for annotator in annotators:

  with open('/content/drive/Shareddrives/FYP-CodeStars/Annotations/'+annotator+'/TypeSystem.xml', 'rb') as f:
    typesystem = load_typesystem(f)

  with open('/content/drive/Shareddrives/FYP-CodeStars/Annotations/'+annotator+'/Full_Kappa_Annotation_'+annotator+'.xmi', 'rb') as f:
    doc = load_cas_from_xmi(f, typesystem=typesystem)

  # count = 0;
  for (index, sentence) in enumerate(doc.select('webanno.custom.'+tag_set)):
    # print(f"{sentence.get_covered_text()}\t{sentence.Humor}")
    for (tag_id, tag) in enumerate(tags):
      if (sentence.Sentiment==tag): #Change to tag name
        # print(f"{sentence.get_covered_text()}\t{sentence.HateSpeech}")
        matrix_to_kappa[index,tag_id]=matrix_to_kappa[index,tag_id]+1
        # count = count+1;
    
  # print(count)
print(matrix_to_kappa)

[[0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 3 0 0]
 [0 0 3 0]
 [0 0 3 0]
 [2 0 1 0]
 [0 0 3 0]
 [3 0 0 0]
 [0 3 0 0]
 [0 3 0 0]
 [0 3 0 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 3 0 0]
 [1 0 2 0]
 [0 0 3 0]
 [0 3 0 0]
 [0 2 1 0]
 [3 0 0 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 3 0 0]
 [0 3 0 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 1 2 0]
 [0 3 0 0]
 [0 2 1 0]
 [0 2 1 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 1 2 0]
 [0 3 0 0]
 [0 3 0 0]
 [0 3 0 0]
 [0 0 3 0]
 [0 3 0 0]
 [0 2 1 0]
 [0 1 2 0]
 [0 3 0 0]
 [0 0 3 0]
 [0 3 0 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [1 0 2 0]
 [0 0 3 0]
 [0 3 0 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 3 0 0]
 [3 0 0 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 3 0 0]
 [0 2 1 0]
 [1 0 2 0]
 [0 0 3 0]
 [0 3 0 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 1 2 0]
 [0 0 3 0]
 [0 3 0 0]
 [0 0 3 0]
 [0 0 3 0]
 [0 3 0 0]
 [0 3 0 0]
 [0 3 0 0]
 [0 0 3 0]
 [0 3 0 0]
 [0 0 3 0]
 [0 3 0 0]
 [0 3 0 0]

## **Calculate Inter Annotation Agreement**

---



#### **Calculate Cohen Kappa** 

In [243]:
# import sklearn.metrics
# sklearn.metrics.cohen_kappa_score(['humorous','non-humorous','humorous'], ['humorous','non-humorous',''], labels=None, weights=None, sample_weight=None)

#### **Calculate Fleiss Kappa**

Reference: https://towardsdatascience.com/inter-annotator-agreement-2f46c6d37bf3

In [244]:
def fleiss_kappa(M):
    """Computes Fleiss' kappa for group of annotators.
    :param M: a matrix of shape (:attr:'N', :attr:'k') with 'N' = number of subjects and 'k' = the number of categories.
        'M[i, j]' represent the number of raters who assigned the 'i'th subject to the 'j'th category.
    :type: numpy matrix
    :rtype: float
    :return: Fleiss' kappa score
    """
    N, k = M.shape  # N is # of items, k is # of categories
    n_annotators = float(np.sum(M[0, :]))  # # of annotators
    tot_annotations = N * n_annotators  # the total # of annotations
    category_sum = np.sum(M, axis=0)  # the sum of each category over all items

    # chance agreement
    p = category_sum / tot_annotations  # the distribution of each category over all annotations
    PbarE = np.sum(p * p)  # average chance agreement over all categories

    # observed agreement
    P = (np.sum(M * M, axis=1) - n_annotators) / (n_annotators * (n_annotators - 1))
    Pbar = np.sum(P) / N  # add all observed agreement chances per item and divide by amount of items

    return round((Pbar - PbarE) / (1 - PbarE), 4)

fleiss_kappa(matrix_to_kappa)

0.7898

In [245]:
import statsmodels
from statsmodels.stats.inter_rater import fleiss_kappa
fleiss_kappa(np.array(matrix_to_kappa))

0.7897574123989217